In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
import scipy.io
from torchvision import datasets, transforms
import torchvision.models as models

In [2]:
class Self_Attention(nn.Module):
    def __init__(self, in_channels,out_channels):
        super(Self_Attention, self).__init__()
        self.query_matrix = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.key_matrix = nn.Conv2d(in_channels, out_channels , kernel_size=1)
        self.value_matrix = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.final_conv=nn.Conv2d(out_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        batch_size, channels, height, width = x.size()
        query_vec=self.query_matrix(x)
        query_vec=query_vec.view(batch_size,-1,height*width)
        key_vec =self.key_matrix(x)
        key_vec = key_vec.view(batch_size,-1,height*width)
        value_vec =self.value_matrix(x)
        value_vec = value_vec.view(batch_size,-1,height*width)
        similarities=torch.bmm(query_vec.permute(0,2,1),key_vec)
        attention_scores=F.softmax(similarities,dim=1)
        output=torch.bmm(value_vec,attention_scores)
        output=output.view(batch_size,-1,height,width)
        output=self.final_conv(output)
        output=self.gamma*output+x
        return output


In [3]:
class CNNWithAttention(nn.Module):
    def __init__(self):
        super(CNNWithAttention, self).__init__()
        self.conv_layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1,stride=3)
        self.self_atten_layer1 = Self_Attention(32,4)
        self.conv_layer2 = nn.Conv2d(32, 64, kernel_size=3, padding=1,stride=3)
        self.self_atten_layer2 = Self_Attention(64,16)
        self.conv_layer3 = nn.Conv2d(64, 128, kernel_size=3, padding=1,stride=3)
        self.self_atten_layer3 = Self_Attention(128,32)
        self.conv_layer4 = nn.Conv2d(128, 256, kernel_size=3, padding=1,stride=3)
        self.self_atten_layer4 = Self_Attention(256,64)
        self.conv_layer5 = nn.Conv2d(256, 10, kernel_size=3, padding=1,stride=3)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        x=self.conv_layer1(x)
        x=self.self_atten_layer1(x)
        x=self.conv_layer2(x)
        x=self.self_atten_layer2(x)
        x=self.conv_layer3(x)
        x=self.self_atten_layer3(x)
        x=self.conv_layer4(x)
        x=self.self_atten_layer4(x)
        x=self.conv_layer5(x)
        x=self.global_avg_pool(x)
        x = x.view(-1,10 )
        return F.log_softmax(x, dim=1)

In [5]:
# Load .mat data from Google Drive
train_data_path = '/content/drive/MyDrive/Colab Notebooks/cifar-10-batches-mat/data_batch_1.mat'
mat_data = scipy.io.loadmat(train_data_path)

# Extract relevant data and reshape
train_data_X = torch.tensor(mat_data["data"]).reshape(-1, 3, 32, 32).float() / 255.0  # Reshape and normalize
train_data_Y = torch.tensor(mat_data["labels"]).squeeze()  # Squeeze to remove extra dimension

# Create TensorDataset
train_dataset = TensorDataset(train_data_X, train_data_Y)

# Create DataLoader
batch_size = 100  # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# train_loader = train_loader.to('cuda')


In [6]:
# Instantiate the model
model = CNNWithAttention()
# model = model.to('cuda')
# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
tot_loss=0
# Training loop
for epoch in range(5):  # Adjust the number of epochs as needed
    model.train()
    tot_loss=0
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tot_loss=+loss.item()
        # print(batch_idx," batch loss:-",loss.item())
    print(tot_loss)

1.9204156398773193
1.9103448390960693
1.764265537261963
1.6732937097549438
1.5931392908096313


In [7]:
# Load .mat data from Google Drive
test_data_path = '/content/drive/MyDrive/Colab Notebooks/cifar-10-batches-mat/test_batch.mat'
test_mat_data = scipy.io.loadmat(test_data_path)


# Extract relevant data and reshape
test_data_X = torch.tensor(test_mat_data["data"]).reshape(-1, 3, 32, 32).float() / 255.0  # Reshape and normalize
test_data_Y = torch.tensor(test_mat_data["labels"]).squeeze()  # Squeeze to remove extra dimension
print(test_data_X.shape)
# Create TensorDataset
test_dataset = TensorDataset(test_data_X, test_data_Y)

# Create DataLoader
batch_size = 100  # Adjust batch size as needed
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

torch.Size([10000, 3, 32, 32])


In [8]:
# Evaluate the model
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        # data, target = data.to('cuda'), target.to('cuda')
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0166, Accuracy: 4209/10000 (42.09%)

